In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from rdkit import Chem
from tensorflow.keras.callbacks import EarlyStopping
from megnet.data.molecule import MolecularGraph
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from sklearn.preprocessing import StandardScaler

C:\Users\quche\anaconda3\envs\ml\lib\site-packages\pymatgen\core\__init__.py:49: UserWarning: Error loading .pmgrc.yaml: [Errno 2] No such file or directory: 'C:\\Users\\quche\\.pmgrc.yaml'. You may need to reconfigure your yaml file.
  warnings.warn(f"Error loading .pmgrc.yaml: {ex}. You may need to reconfigure your yaml file.")


## Load the original dataset and compute the mean and variance of the original data set, in order to transform the predicted value back to Kelvin

In [3]:
data = pd.read_csv('data/bp_new3.csv', delim_whitespace=True)
y = data['bp'].to_numpy().reshape((-1,1))
scaler = StandardScaler().fit(y)
y = scaler.transform(y)

## Create the model and load saved weights

In [4]:
graph = MolecularGraph(atom_features=['element', 'hybridization', 'formal_charge'],
                       bond_features=['bond_type', 'same_ring', 'graph_distance'],
                       known_elements=['C', 'H', 'O', 'N', 'F', 'Si', 'S', 'Cl', 'Br', 'P', 'I'],
                       cutoff = 5)

model = MEGNetModel(nfeat_edge=7, nfeat_global=3, nfeat_node=18, nblocks=3, learning_rate=2e-4,
                    n1=64, n2=32, n3=16, npass=2, ntarget=1, act='relu', final_act=None,
                    graph_converter=graph, loss="mae")

model.load_weights('./checkpoints/bp_model_weights')

## Use the model to make predictions

In [5]:
mol = Chem.MolFromSmiles("CCCO")
bp = model.predict_structure(mol)
y_pred = np.array(bp).reshape(-1,1)
y_pred = scaler.inverse_transform(y_pred)
print("Predicted Boiling Point (K): ", y_pred[0,0])

Predicted Boiling Point (K):  373.79205
